<a href="https://colab.research.google.com/github/abiolaTresor/Disaster-Or-Not/blob/abiola/data_treatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install transformers

In [34]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
import random
from transformers import BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
import time
import datetime
#import logging
#import matplotlib.pyplot as plt
#import seaborn as sns
#from sklearn.model_selection import train_test_split
# keras.preprocessing.sequence import pad_sequences
#from torch.utilis.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
#import time

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("there are %d GPU(s) available." % torch.cuda.device_count())
  print("The GPU used in this code is: ", torch.cuda.get_device_name(0))
else:
  assert False, "No GPU detected. Please, select GPU in the Colab."

there are 1 GPU(s) available.
The GPU used in this code is:  Tesla P100-PCIE-16GB


In [37]:
df = pd.read_csv("/content/drive/My Drive/Disaster-Or-Not/data/train.csv", delimiter=",", header=0, names=["id", "keyword", "location", "text","target"])

In [38]:
df.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [39]:
print("dimensions of training data:", df.shape)

dimensions of training data: (7613, 5)


In [40]:
print("the number of sentences seen as disaster telling:", df[df.target==1].shape[0])

the number of sentences seen as disaster telling: 3271


In [41]:
print("the number of observations of keyword == naN is:",df[pd.isna(df.keyword)].shape[0])

the number of observations of keyword == naN is: 61


In [42]:
df[df.target == 1].sample(5)[['text']]

,text
7270,I moved to England five years ago today. What ...
4597,CLEARED:incident with injury:I-495 inner loop...
3962,When the waves are flooding the shore\nAnd I c...
7518,CIA plot! *rolling eyes* RT @ajabrown: Chinese...
4748,The Lightning out here is something serious!


In [43]:
texts = df.text.values
targets = df.target.values

In [44]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [45]:
print(texts[150])
print(tokenizer.tokenize(texts[150]))

@mickinyman @TheAtlantic That or they might be killed in an airplane accident in the night a car wreck! Politics at it's best.
['@', 'mick', '##iny', '##man', '@', 'the', '##at', '##lan', '##tic', 'that', 'or', 'they', 'might', 'be', 'killed', 'in', 'an', 'airplane', 'accident', 'in', 'the', 'night', 'a', 'car', 'wreck', '!', 'politics', 'at', 'it', "'", 's', 'best', '.']


In [46]:
max_len = 0

for text in texts:
    input_ids = tokenizer.encode(text, add_special_tokens=True)
    max_len = max(max_len,len(input_ids))
print("max length to set to is", max_len)

max length to set to is 84


In [47]:
input_ids = []
attention_masks = []

for text in texts:
    encoded_dict = tokenizer.encode_plus(text, add_special_tokens=True, max_length=84, truncation=True, pad_to_max_length=True, return_attention_mask=True, return_tensors='pt')
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids,dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
targets = torch.tensor(targets)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [48]:
dataset = TensorDataset(input_ids, attention_masks, targets)

train_size = int(0.9 * len(dataset))
val_size = len(dataset)-train_size

random.seed(10)
train_dataset, val_dataset = random_split(dataset,[train_size,val_size])

In [49]:

batch_size = 32

train_data_loader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size= batch_size)
validation_data_loader = DataLoader(val_dataset, sampler = SequentialSampler(val_dataset), batch_size = batch_size)

In [50]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2, output_attentions = False, output_hidden_states = False)

model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [51]:
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)
epochs = 4

total_steps = len(train_data_loader)*epochs

scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0, num_training_steps = total_steps)

In [52]:
def flat_accuracy (preds, labels):
    pred_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat)/len(labels_flat)

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round(elapsed))))

In [56]:


random.seed(32)
np.random.seed(32)
torch.manual_seed(32)


#Storing training stats
training_stats = []

total_t0 = time.time()


for epoch_i in range(0, epochs):
    print("")
    print("================ Epoch {:} / {:} ================".format(epoch_i + 1, epochs))
    print('Training ongoing...')

    t0 = time.time()

    #reseting total loss for current epoch
    total_train_loss = 0

    #Activating train mode
    model.train

    for step, batch in enumerate(train_data_loader):
        
        # Progress visualization every 15 batches
        if step % 15 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print(' Batch {: >5,} of {:>5,}.  Elapsed: {:}.'.format(step, len(train_data_loader), elapsed))
        
        # Unpacking batches
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        #Clear Grads
        model.zero_grad()

        #performing forward pass
        loss, loggits = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask, labels= b_labels)

        #Accumulating training loss
        total_train_loss += loss.item()

        # Performing Backward pass
        loss.backward()

        # Setting the norm of gradients to 1.0.
        torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)

        #Updating parameters
        optimizer.step()

        #Updating the learning rate
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_data_loader)
    training_time = format_time(time.time() - t0)

    print("")
    print(" Avg training loss: {0:.2f}".format(avg_train_loss))
    print(" Training epoch took: {:}".format(training_time)) 

    print("Running validation")

    t0 = time.time()

    model.eval()


    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_data_loader:

      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)

      with torch.no_grad():

        (loss, logits) = model(b_input_ids,token_type_ids = None, attention_mask = b_input_mask, labels = b_labels)

      total_eval_loss += loss.item()

      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

      total_eval_accuracy += flat_accuracy(logits, label_ids)

    print("validation Over")

    avg_val_accuracy = total_eval_loss/len(validation_data_loader)
    avg_val_loss = total_eval_loss/len(validation_data_loader)
    validation_time = format_time(time.time()-t0)

    print(" Accuracy: {0:.2f}".format(avg_val_accuracy))
    print(" Validation Loss {0:.2f}".format(avg_val_loss))
    print(" Validation took: {:}".format(validation_time))

    training_stats.append({"epoch": epoch_i + 1,
                         "training_loss": avg_train_loss,
                         "val_loss": avg_val_loss,
                         "val_accuracy": avg_val_accuracy,
                         "training_time": training_time,
                         "validation_time": validation_time
                         })

print("Training Over!!!!")
print("it took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



================ Epoch 1 / 4 ================
Training ongoing...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


 Batch    15 of   215.  Elapsed: 0:00:05.
 Batch    30 of   215.  Elapsed: 0:00:09.
 Batch    45 of   215.  Elapsed: 0:00:14.
 Batch    60 of   215.  Elapsed: 0:00:18.
 Batch    75 of   215.  Elapsed: 0:00:23.
 Batch    90 of   215.  Elapsed: 0:00:27.
 Batch   105 of   215.  Elapsed: 0:00:32.
 Batch   120 of   215.  Elapsed: 0:00:36.
 Batch   135 of   215.  Elapsed: 0:00:41.
 Batch   150 of   215.  Elapsed: 0:00:45.
 Batch   165 of   215.  Elapsed: 0:00:50.
 Batch   180 of   215.  Elapsed: 0:00:54.
 Batch   195 of   215.  Elapsed: 0:00:58.
 Batch   210 of   215.  Elapsed: 0:01:03.

 Avg training loss: 0.45
 Training epoch took: 0:01:04
Running validation
validation Over
 Accuracy: 0.40
 Validation Loss 0.40
 Validation took: 0:00:02

================ Epoch 2 / 4 ================
Training ongoing...
 Batch    15 of   215.  Elapsed: 0:00:04.
 Batch    30 of   215.  Elapsed: 0:00:09.
 Batch    45 of   215.  Elapsed: 0:00:13.
 Batch    60 of   215.  Elapsed: 0:00:18.
 Batch    75 of   215.